In [1]:
# Load the pickle file
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import numpy as np
import pandas as pd

2025-04-07 12:57:23.333323: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# load the trained ANN model, the scaler and the one-hot encoder and the label encoder

model = load_model('model.h5') 

# load the encder and scaler

with open('geo_one_hot_encoder.pkl','rb') as file:
    geo_one_hot_encoder = pickle.load(file)

with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender = pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler = pickle.load(file)

    

In [3]:
# Example input data

input_data = {
    'CreditScore': 300,
    'Geography':'France',
    'Gender':'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000

}

In [ ]:
# one hot encode the geography
# Convert the input data to a DataFrame and encoding all data so that it can be passed to the scaler and model
geo_encoded = geo_one_hot_encoder.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=geo_one_hot_encoder.get_feature_names_out(['Geography']))
geo_encoded_df

/Users/abdulqavi/Documents/analysis/venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [18]:
# Create a DataFrame from input data 
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,300,France,Male,40,3,60000,2,1,1,50000


In [19]:
# Concatenate the one-hot encoded geography with the input DataFrame
input_df = pd.concat([input_df.drop(columns=['Geography'], axis=1),geo_encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,300,Male,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [20]:
# encode the gender categorical variable
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,300,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [21]:
# one more transformation to make sure the input is in the right format
# scale the input data
input_df_scaled = scaler.transform(input_df)
input_df_scaled

array([[-3.54326899,  0.90911166,  0.09477172, -0.69844549, -0.29010416,
         0.80510537,  0.63367318,  0.95214374, -0.84805047,  0.98019606,
        -0.57581067, -0.56349184]])

In [23]:
# predict the churn
prediction = model.predict(input_df_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step


array([[0.11390544]], dtype=float32)

In [29]:
prediction_prob = prediction[0][0]
prediction_prob

0.113905445

In [30]:
if prediction_prob > 0.5:
    print("The customer is likely to churn.")
else:
    print("The customer is not likely to churn.")

The customer is not likely to churn.


In [31]:
geo_one_hot_encoder.categories_


[array(['France', 'Germany', 'Spain'], dtype=object)]

In [32]:
label_encoder_gender.classes_

array(['Female', 'Male'], dtype=object)